In [ ]:
# Plot predicted scores against true scores with the base thresholds for converting score to sii
sns.set_theme(style="white")
fig, axes = plt.subplots(1, 5, figsize=(14, 6))

scatter1 = axes[0].scatter(train['PCIAT-PCIAT_Total'], oof_lgb, c=train["sii"], cmap="autumn", alpha=0.5)
axes[0].set_xlabel("True Score")
axes[0].set_ylabel("OOF Predictions - LGBM")
axes[0].set_ylim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[0].set_xlim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[0].set_aspect('equal', adjustable='box')

thresholds = [30, 50, 80]
for threshold in thresholds:
    axes[0].axhline(threshold, color="blue", linestyle="--", lw=1)
    axes[0].axvline(threshold, color="blue", linestyle="--", lw=1)

scatter2 = axes[1].scatter(train['PCIAT-PCIAT_Total'], oof_xgb, c=train["sii"], cmap="autumn", alpha=0.5)
axes[1].set_xlabel("True Score")
axes[1].set_ylabel("OOF Predictions - XGB")
axes[1].set_ylim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[1].set_xlim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[1].set_aspect('equal', adjustable='box')

for threshold in thresholds:
    axes[1].axhline(threshold, color="blue", linestyle="--", lw=1)
    axes[1].axvline(threshold, color="blue", linestyle="--", lw=1)

scatter2 = axes[2].scatter(train['PCIAT-PCIAT_Total'], oof_xgb_2, c=train["sii"], cmap="autumn", alpha=0.5)
axes[2].set_xlabel("True Score")
axes[2].set_ylabel("OOF Predictions - XGB (2)")
axes[2].set_ylim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[2].set_xlim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[2].set_aspect('equal', adjustable='box')

for threshold in thresholds:
    axes[2].axhline(threshold, color="blue", linestyle="--", lw=1)
    axes[2].axvline(threshold, color="blue", linestyle="--", lw=1)
    
scatter3 = axes[3].scatter(train['PCIAT-PCIAT_Total'], oof_cat, c=train["sii"], cmap="autumn", alpha=0.5)
axes[3].set_xlabel("True Score")
axes[3].set_ylabel("OOF Predictions - Cat")
axes[3].set_ylim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[3].set_xlim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[3].set_aspect('equal', adjustable='box')

for threshold in thresholds:
    axes[3].axhline(threshold, color="blue", linestyle="--", lw=1)
    axes[3].axvline(threshold, color="blue", linestyle="--", lw=1)
    
scatter3 = axes[4].scatter(train['PCIAT-PCIAT_Total'], oof_xtrees, c=train["sii"], cmap="autumn", alpha=0.5)
axes[4].set_xlabel("True Score")
axes[4].set_ylabel("OOF Predictions - ExtraTrees")
axes[4].set_ylim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[4].set_xlim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[4].set_aspect('equal', adjustable='box')

for threshold in thresholds:
    axes[4].axhline(threshold, color="blue", linestyle="--", lw=1)
    axes[4].axvline(threshold, color="blue", linestyle="--", lw=1)

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1,3, figsize=(20,6))

model_preds = pd.DataFrame({
    'lgb': oof_lgb,
    'xgb': oof_xgb,
    'xgb2': oof_xgb_2,
    'cat': oof_cat,
    'xtrees': oof_xtrees
})
corr_df = model_preds.corr()
sns.heatmap(corr_df, annot=True, cmap="autumn", cbar=False, linewidths=0.5, linecolor='black', ax=axes[0])
axes[0].set_title("Correlation Between Models")

axes[1].scatter(
    train['PCIAT-PCIAT_Total'], np.average(model_preds, axis=1, weights=[0.2, 0.2, 0.3, 0.1, 0.2]), c=train["sii"], cmap="autumn", alpha=0.5
)
axes[1].set_xlabel("True Score")
axes[1].set_ylabel("OOF Predictions - Ensemble")
axes[1].set_ylim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[1].set_xlim(0,np.max(train['PCIAT-PCIAT_Total']))
axes[1].set_aspect('equal', adjustable='box')
for threshold in thresholds:
    axes[1].axhline(threshold, color="blue", linestyle="--", lw=1)
    axes[1].axvline(threshold, color="blue", linestyle="--", lw=1)


meta_model = LinearRegression(fit_intercept = False)
meta_model.fit(model_preds, train['PCIAT-PCIAT_Total'])
meta_model.coef_
print(f"Coefs 'Meta Model': {meta_model.coef_}")

lgb_thresholds_ens = np.mean(np.array(lgb_thresholds), axis=0)
xgb_thresholds_ens = np.mean(np.array(xgb_thresholds), axis=0)
xgb_2_thresholds_ens = np.mean(np.array(xgb_2_thresholds), axis=0)
cat_thresholds_ens = np.mean(np.array(cat_thresholds), axis=0)
xtrees_thresholds_ens = np.mean(np.array(xtrees_thresholds), axis=0)

thresholds_df = pd.DataFrame({
    "LGB Thresholds": lgb_thresholds_ens,
    "XGB Thresholds": xgb_thresholds_ens,
    "XGB 2 Thresholds": xgb_2_thresholds_ens,
    "Cat Thresholds": cat_thresholds_ens,
    "Xtrees Thresholds": xtrees_thresholds_ens
})

sns.heatmap(thresholds_df, annot=True, cmap="autumn", cbar=False, linewidths=0.5, linecolor='black', ax=axes[2])
axes[2].set_title("Ensemble Thresholds Derived from CV")
axes[2].set_xticklabels(thresholds_df.columns, rotation=45)  
plt.show()

In [ ]:
# Showing how sensitive QWK is with changes in threshold
scores = []
ts = []
m = 30
for i in tqdm(np.linspace(-1,10, 100)):
    thresholds = [m+i, 50, 80]
    pred = round_with_thresholds(oof_xgb, thresholds)
    score = cohen_kappa_score(train["sii"], pred, weights='quadratic')
    ts.append(m+i)
    scores.append(score)
    
plt.plot(ts, scores)
plt.title("Demonstration of QWK sensitivity to changes in threshold 0")
plt.show()

In [ ]:
lgb_importances = pd.DataFrame({
    'Feature': lgb_features,
    'Importance': lgb_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

xgb_importances = pd.DataFrame({
    'Feature': xgb_features,
    'Importance': xgb_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

cat_importances = pd.DataFrame({
    'Feature': cat_features,
    'Importance': cat_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Set the number of features to display
n_top_features = 40

fig, axes = plt.subplots(1, 3, figsize=(18, 8))
sns.set_theme(style="whitegrid")

sns.barplot(ax=axes[0], data=lgb_importances.head(n_top_features),
            x='Importance', y='Feature', palette="autumn")
axes[0].set_title('LightGBM Top Feature Importances')

sns.barplot(ax=axes[1], data=xgb_importances.head(n_top_features),
            x='Importance', y='Feature', palette="autumn")
axes[1].set_title('XGBoost Top Feature Importances')

sns.barplot(ax=axes[2], data=cat_importances.head(n_top_features),
            x='Importance', y='Feature', palette="autumn")
axes[2].set_title('CatBoost Top Feature Importances')

plt.tight_layout()
plt.show()

In [2]:
(set(lgb_importances[-10:]["Feature"]) & set(xgb_importances[-10:]["Feature"]) & set(cat_importances[-10:]["Feature"]))

In [ ]:
# Apply the optimized thresholds to OOF predictions
oof_lgb = round_with_thresholds(oof_lgb, lgb_thresholds_ens)
print(f"LGBM optimized Kappa: {cohen_kappa_score(train['sii'], oof_lgb, weights='quadratic')}")
oof_xgb = round_with_thresholds(oof_xgb, xgb_thresholds_ens)
print(f"XGB optimized Kappa: {cohen_kappa_score(train['sii'], oof_xgb, weights='quadratic')}")
oof_xgb_2 = round_with_thresholds(oof_xgb_2, xgb_2_thresholds_ens)
print(f"XGB2 optimized Kappa: {cohen_kappa_score(train['sii'], oof_xgb_2, weights='quadratic')}")
oof_cat = round_with_thresholds(oof_cat, cat_thresholds_ens)
print(f"CAT optimized Kappa: {cohen_kappa_score(train['sii'], oof_cat, weights='quadratic')}")
oof_xtrees = round_with_thresholds(oof_xtrees, xtrees_thresholds_ens)
print(f"ExtraTrees optimized Kappa: {cohen_kappa_score(train['sii'], oof_xtrees, weights='quadratic')}")

if voting:
    oof_preds = np.array([oof_lgb, oof_xgb, oof_xgb_2, oof_cat, oof_xtrees])
    voted_oof = stats.mode(oof_preds, axis=0).mode.flatten().astype(int)
    final_oof = voted_oof
else: 
    weights = [0.2, 0.2, 0.3, 0.1, 0.2]
    oof_preds = np.array([oof_lgb, oof_xgb, oof_xgb_2, oof_cat, oof_xtrees])
    weighted_oof = np.average(oof_preds, axis=0, weights=weights)
    final_oof = np.round(weighted_oof).astype(int)

pd.DataFrame(final_oof).to_csv("oof_mv41.csv")
# Calculate Kappa score for voted OOF predictions
kappa_score = cohen_kappa_score(train["sii"], final_oof, weights='quadratic')
print(f"Ensemble Kappa score: {kappa_score}")
# Plot confusion matrix
conf_matrix = confusion_matrix(train["sii"], final_oof)
sns.set_theme(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="autumn", cbar=False, linewidths=0.5, linecolor='black')
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.show()

In [ ]:
# Apply the optimized thresholds to test predictions
test_lgb = round_with_thresholds(test_lgb, lgb_thresholds_ens)
test_xgb = round_with_thresholds(test_xgb, xgb_thresholds_ens)
test_xgb_2 = round_with_thresholds(test_xgb_2, xgb_2_thresholds_ens)
test_cat = round_with_thresholds(test_cat, cat_thresholds_ens)
test_xtrees = round_with_thresholds(test_xtrees, xtrees_thresholds_ens)

if voting:
    test_preds = np.array([test_lgb, test_xgb, test_xgb_2, test_cat, test_xtrees])
    voted_test = stats.mode(test_preds, axis=0).mode.flatten().astype(int)
    final_test = np.round(voted_test).astype(int)
else:
    test_preds = np.array([test_lgb, test_xgb, test_xgb_2, test_cat, test_xtrees])
    weighted_test = np.average(test_preds, axis=0, weights=weights)
    final_test = np.round(weighted_test).astype(int)